## Pset 1

### Problem 1

### Problem 2

### Problem 3

#### Part 1

The final goods in this economy are instant noodles (600 dollars) and the portion of wheat sold as a final good. The wheat company's total revnue is 250 dollars, but some of that revenue is sold to the noodle company as an intermediate good (100 dollars). Thus, the wheat is sold as a final good at 250 - 100 = 150 dollars.

The GDP is the value of the final goods (150 dollars for wheat and 600 dollars for instant noodles), which amounts to 750 dollars.